# A Julia Note at Colab

This notebook gives instructions for getting started with using Julia at Colab. The basic issues are: (1) how to make Colab aware of my files located at "My Drive" (2) how to load packages and (3) how to load a local module?

## Mounting My Drive

There is currently no Julia command for this, but can be done in Python. Therefore, change the "Runtime type" (see "Connect" at the top right menu bar) to Python. Run the next cell, and then change back to Julia.

In [1]:
from google.colab import drive                #do this in Python, then switch to Julia
drive.mount('/content/drive')

Mounted at /content/drive


## Import Packages

Colab has a very short list of installed packages, so you probably have to add some. This has (currently) to be done for each session. For the ?
`FinEcmt_OLS` module, you need the packes added in the next cell.

In [1]:
import Pkg
Pkg.add(["Distributions","StatsBase","FiniteDiff"])

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed FiniteDiff ─ v2.27.0
    Updating `~/.julia/environments/v1.10/Project.toml`
  [31c24e10] + Distributions v0.25.118
  [6a86dc24] + FiniteDiff v2.27.0
  [2913bbd2] + StatsBase v0.34.4
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [6a86dc24] + FiniteDiff v2.27.0
Precompiling packages...
   2644.3 ms  ✓ FiniteDiff
   2109.5 ms  ✓ FiniteDiff → FiniteDiffSparseArraysExt
   2264.9 ms  ✓ FiniteDiff → FiniteDiffStaticArraysExt
  3 dependencies successfully precompiled in 16 seconds. 460 already precompiled.


# Load the `FinEcmt_OLS` module. It is on "My Drive"

Then load some extra packages that actually are installed on Colab.

In [2]:
myfolder = "/content/drive/My Drive/Test"         #Julia code
readdir(myfolder)

include(joinpath(myfolder,"src","FinEcmt_OLS.jl"))
using .FinEcmt_OLS

In [3]:
using Statistics, DelimitedFiles, LinearAlgebra

# Start Working...

In [4]:
x = readdlm(joinpath(myfolder,"Data/FFmFactorsPs.csv"),',',skipstart=1)

                #yearmonth, market, small minus big, high minus low
(ym,Rme,RSMB,RHML) = (x[:,1],x[:,2]/100,x[:,3]/100,x[:,4]/100)
x = nothing

printlnPs("Sample size:",size(Rme))

Sample size:    (388,)


In [5]:
@doc2 OlsGM

```
OlsGM(Y,X)
```

LS of Y on X; for one dependent variable, Gauss-Markov assumptions

### Input

  * `Y::Vector`:    T-vector, the dependent variable
  * `X::Matrix`:    Txk matrix of regressors (including deterministic ones)

### Output

  * `b::Vector`:    k-vector, regression coefficients
  * `u::Vector`:    T-vector, residuals Y - yhat
  * `Yhat::Vector`: T-vector, fitted values X*b
  * `V::Matrix`:    kxk matrix, covariance matrix of b
  * `R²::Number`:   scalar, R² value


## OLS Regression

In [6]:
Y = Rme                    #to get standard OLS notation
T = size(Y,1)
X = [ones(T) RSMB RHML]

(b,_,_,V,R²) = OlsGM(Y,X)
Stdb = sqrt.(diag(V))        #standard errors

printblue("OLS Results:\n")
xNames = ["c","SMB","HML"]
printmat(b,Stdb,colNames=["b","std"],rowNames=xNames)

printlnPs("R²: ",R²)

OLS Results:

            b       std
c       0.007     0.002
SMB     0.217     0.073
HML    -0.429     0.074

      R²:      0.134


In [7]:
RegressionTable(b,V,["c","SMB","HML"])     #a function for printing regression results

         coef    stderr    t-stat   p-value
c       0.007     0.002     3.175     0.001
SMB     0.217     0.073     2.957     0.003
HML    -0.429     0.074    -5.836     0.000

